In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import datetime
from datetime import datetime as dt
import os
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

#### Assume that we save all files in all/ folder in the same directory 

In [6]:
# Utils 
def lag_categorize_feature(data, prefix = ''):
    if data > 0: 
        if data > 5:
            return ('highly_decreasing{}'.format(prefix))
        else:
            return ('decreasing{}'.format(prefix))
    elif data == 0 :
        return ('stable{}'.format(prefix))
    elif data < 0 :
        if data < -5: 
            return ('highly_increasing{}'.format(prefix))
        else: 
            return ('increasing{}'.format(prefix))
    else: 
        return ('No_info{}'.format(prefix))
    
def weekday_weekend_clf(data):
    if data in (5, 6):
        return 'weekend'
    else:
        return 'weekday'
    
    
from IPython.display import Audio, display

def allDone():
  display(Audio(url='https://sound.peal.io/ps/audios/000/000/537/original/woo_vu_luvub_dub_dub.wav', autoplay=True))
# Insert whatever audio file you want above    

In [21]:
def read_train(folder):
    df = pd.read_csv('all/train.csv')
    df['fdt'] = pd.to_datetime(df['first_active_month'])
    df['yr'] = df.fdt.apply(lambda x: x.year)
    df['tenure'] = ((datetime.date(2018, 2, 1) - df['fdt'].dt.date).dt.days)/30
    return df

def read_test(folder):
    df = pd.read_csv('all/test.csv')
    df['fdt'] = pd.to_datetime(df['first_active_month'])
    df['yr'] = df.fdt.apply(lambda x: x.year)
    df['tenure'] = ((datetime.date(2018, 2, 1) - df['fdt'].dt.date).dt.days)/30
    return df

def read_txns(fname):
    df = pd.read_csv(fname)
    
    # convert to date time
    df['purchase_date'] = pd.to_datetime(df.purchase_date)
#     df['purchase_weekday'] = map(weekday_weekend_clf, df.purchase_date.dt.weekday)    
    df['purchase_weekday'] = df.purchase_date.dt.weekday
    df['purchase_month'] = df['purchase_date'].dt.month
    df['purchase_firstday'] = df['purchase_date'].dt.day==1
    # binary 
    for col in ['authorized_flag', 'category_1']:
        df[col] = df[col].map({'Y':1, 'N':0})
    

    return df

    
    
def read_merchant(fname):
    # Since category column exist in transaction data already, we filter them out before merge into transaction data 
    merchant = pd.read_csv(fname).drop(['category_1', 'category_2', 'city_id', 'state_id', 'merchant_category_id', 'subsector_id'], 1)
    merchant['sales_lag_3'] = map(lag_categorize_feature, merchant.avg_sales_lag3.values)
    merchant['sales_lag_6'] = map(lag_categorize_feature, merchant.avg_sales_lag6.values)
    merchant['sales_lag_12'] = map(lag_categorize_feature, merchant.avg_sales_lag12.values)
    merchant['purchases_lag3'] = map(lag_categorize_feature, merchant.avg_purchases_lag3.values)
    merchant['purchases_lag6'] = map(lag_categorize_feature, merchant.avg_purchases_lag6.values)
    merchant['purchases_lag12'] = map(lag_categorize_feature, merchant.avg_purchases_lag12.values)
    merchant = pd.get_dummies(merchant, columns= ['sales_lag_3', 
                                                  'sales_lag_6', 
                                                  'sales_lag_12', 
                                                  'purchases_lag3', 
                                                  'purchases_lag6', 
                                                  'purchases_lag12',
                                                  'most_recent_sales_range', 
                                                  'most_recent_purchases_range'])
    return merchant 

def agg_tx_features(df, prefix=""):
    df.purchase_weekday = map(weekday_weekend_clf, df.purchase_weekday)    
    df =  pd.get_dummies(df, columns=['purchase_weekday'
                                       ,'category_1'
                                       , 'category_2'
                                       , 'category_3'])
    agg = df.groupby(by='card_id').agg(
        {'purchase_amount': ['count','sum','mean','min','max', 'var', 'skew']
         ,'merchant_id': ['nunique']
         ,'installments': ['sum']
         ,'authorized_flag': ['mean']
         ,'category_1_0': ['count', 'mean']
         ,'category_1_1': ['count', 'mean']         
         ,'category_2_1.0': ['count', 'mean']
         ,'category_2_2.0': ['count', 'mean']
         ,'category_2_3.0': ['count', 'mean']
         ,'category_2_4.0': ['count', 'mean']
         ,'category_2_5.0': ['count', 'mean']
         ,'category_3_A': ['count', 'mean']
         ,'category_3_B': ['count', 'mean']
         ,'category_3_C': ['count', 'mean']
         ,'state_id': ['nunique']
         ,'city_id': ['nunique']
         ,'purchase_month': ['nunique']
         ,'month_lag': ['nunique', 'min', 'max']
         ,'purchase_firstday': ['sum', 'mean']  
         , 'merchant_category_id': ['nunique']
         , 'most_recent_sales_range_A': ['sum', 'mean']
         , 'most_recent_sales_range_B': ['sum', 'mean']
         , 'most_recent_sales_range_C': ['sum', 'mean']
         , 'most_recent_sales_range_D': ['sum', 'mean']
         , 'most_recent_sales_range_E': ['sum', 'mean']
         , 'most_recent_purchases_range_A' : ['sum', 'mean']
         , 'most_recent_purchases_range_B' : ['sum', 'mean']
         , 'most_recent_purchases_range_C' : ['sum', 'mean']
         , 'most_recent_purchases_range_D' : ['sum', 'mean']
         , 'most_recent_purchases_range_E' : ['sum', 'mean']         
         , 'purchases_lag3_highly_decreasing' : ['sum']
         , 'purchases_lag3_decreasing' : ['sum']
         , 'purchases_lag6_highly_decreasing' : ['sum']
         , 'purchases_lag6_decreasing' : ['sum']
         , 'purchases_lag12_highly_decreasing' : ['sum']
         , 'purchases_lag12_decreasing' : ['sum']
        }).reset_index() 
    
    agg.columns = ["card_id"] + [ prefix + '_'.join(tup).rstrip('_') \
                                 for tup in agg.columns.values[1:]]
    return agg

def combine_card_with_transaction(card, txs, min_purchase_date):
    # combine_card_with_transaction
    combine_data = card.merge(txs, how='left', on='card_id')
    combine_data['no_new_tx'] = combine_data.all_purchase_amount_min.isnull()
    combine_data['active_pre_newtx'] = combine_data.fdt <= min_purchase_date
    

    combine_data = combine_data.fillna(0)
    agg_features = list(txs.columns.values)
    agg_features.remove('card_id')
    feat =['yr', 'tenure', 'no_new_tx', 'active_pre_newtx'] + agg_features
    
    # X and y (if exist)
    X = combine_data[feat]
    try: 
        y = combine_data.target
    except: 
        print("No target data, return nan as y.")
        y = np.nan
        
        
    # Return with dict 
    result = {'card_with_txs': agg_features
             , 'featurs': feat
             , 'X': X
             , 'y': y
             }
    
    return result

def evaluate(y, pred):
    res = pd.DataFrame({'y':y, 'pred':pred})
    res['diff'] = res.y-res.pred
    res['ab_diff'] = res['diff'].abs()
    res['diff_sq'] = res['diff']**2
    res['is_outlier'] = res.y < -33
    rmse = np.sqrt(np.mean(res['diff_sq']))
    summary = res.groupby('is_outlier').agg({
        'ab_diff': ['mean', 'min','max', 'median'],
        'y':['count']
    })
    
    return rmse, summary

# 1. Read data

In [17]:
cards = read_train('')


In [6]:
n_txs = read_txns('all/new_merchant_transactions.csv')


In [7]:
h_txs = read_txns('all/historical_transactions.csv')


In [8]:
merchant = read_merchant('all/merchants.csv')

In [9]:
# Combine h_txs + n_txs
txs = h_txs.append(n_txs, ignore_index=True)

In [10]:
del n_txs 
del h_txs 

In [11]:
# combine merchant with transaction data 
txs_with_merchant = pd.merge(txs, merchant, how='left', on = 'merchant_id')

In [13]:
txs_with_merchant.to_csv("txs_with_merchant.csv")

# Features

In [7]:
txs_with_merchant = pd.read_csv('txs_with_merchant.csv')

In [11]:
agg_new = agg_tx_features(txs_with_merchant, "all_")

# Combine aggregate transactions <>  training dataset 

### Train with RandomforestRegressor & grid search CV

In [22]:
agg = combine_card_with_transaction(cards, agg_new, txs_with_merchant.purchase_date.min())


In [45]:
X = agg['X']
y = agg['y']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1)

In [24]:
# clf.train(tr)
clf = RandomForestRegressor(max_depth=10, n_estimators=5000)
clf.fit(X_train, y_train)

# Grid search params -- it takes really long, so we only run the best result now 
# clf_cv = GridSearchCV(clf, {'max_depth': [5, 10], 'n_estimators': [3000, 5000]}, verbose=1, cv = 2)
# clf_cv.fit(X_train,y_train)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=5000, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [29]:
# Write model into file in case memory error. 
path = os.getcwd()
model_file = path + "/clf"

try: 
    with open(model_file, "w+b") as f: # write binary model 
        model = clf 
        pickle.dump(model, f) # write model into file 
    print("Sucessfully write file: {}".format(model_file))
except ValueError:
                print("Error")

Sucessfully write file: /Users/peggy/Desktop/python/Elo/EloMerchantCategory/clf


In [27]:
y_pred = clf.predict(X_test)
perf = evaluate(y_test,y_pred)

print('rmse:' + str(perf[0]))
print(perf[1])

rmse:3.82168443847
                y    ab_diff                                 
            count       mean        min        max     median
is_outlier                                                   
False       39919   1.286793   0.000001  17.236545   0.885253
True          465  31.121332  20.176960  34.833500  31.819749


In [28]:
feature_importances = pd.DataFrame(clf.feature_importances_,
                                   index = X_train.columns,
                                   columns=['importance']).sort_values('importance',ascending=False)

feature_importances

,importance
all_month_lag_min,0.066757
all_month_lag_max,0.065384
tenure,0.064854
all_authorized_flag_mean,0.046576
all_purchase_amount_skew,0.034164
all_category_1_0_mean,0.033221
all_purchase_amount_sum,0.030392
all_installments_sum,0.028458
all_purchase_amount_min,0.022609
all_purchase_amount_mean,0.022595


### Train with xgb & grid search CV

In [46]:
gbm = xgb.XGBRegressor()
# Grid search params
# reg_cv = GridSearchCV(gbm, {"colsample_bytree":[1.0],"min_child_weight":[1.0]
#                             ,'max_depth': [5,10], 'n_estimators': [3000, 5000]}, verbose=1, cv = 2)
# reg_cv.fit(X_train,y_train)
# gbm = xgb.XGBRegressor(**reg_cv.best_params_) # input best params
gbm.fit(X_train,y_train)
y_pred = gbm.predict(X_test)
# gbm.score(X_test, y_test)

perf = evaluate(y_test,y_pred)

print('rmse:' + str(perf[0]))
print(perf[1])

rmse:3.84222694653
                y    ab_diff                                 
            count       mean        min        max     median
is_outlier                                                   
False       39919   1.279830   0.000012  16.868326   0.901875
True          465  31.656774  27.702533  34.174740  31.848056


In [47]:
feature_importances = pd.DataFrame(gbm.feature_importances_,
                                   index = X_train.columns,
                                   columns=['importance']).sort_values('importance',ascending=False)

feature_importances

,importance
all_month_lag_min,0.141618
tenure,0.132948
all_month_lag_max,0.117052
all_authorized_flag_mean,0.109827
all_category_1_1_mean,0.050578
all_month_lag_nunique,0.034682
all_installments_sum,0.030347
all_purchase_amount_skew,0.027457
all_purchase_amount_min,0.023121
all_purchase_amount_max,0.023121


# Predict for new data 

In [30]:
test = read_test('')

In [32]:
#### combine txs <> test data 
test_agg = combine_card_with_transaction(test, agg_new, txs_with_merchant.purchase_date.min())

No target data, return nan as y.


In [48]:
X_test = test_agg['X']

#### RF clf 

In [34]:
y_pred = clf.predict(X_test)
y_pred = pd.DataFrame({'prediction':y_pred})
y_pred.to_csv("y_pred_rf.csv")

#### XGB clf 

In [49]:
y_pred = gbm.predict(X_test)
y_pred = pd.DataFrame({'prediction':y_pred})
y_pred.to_csv("y_pred_xgb.csv")